First we try to implement an ambitious pure-std-Python cellular automata.

In [9]:
# Helper function
import random as rnd

def random_trial(probability):
    'Return True with a given probability, otherwise return False.'
    # Get a random real btw 0 and 1
    selector = rnd.random()
    
    return (selector < probability)
# ---

In [10]:
from functools import reduce

class CellularAutomaton:
    'An extensible cellular automaton.'
    
    # Default location of the neighbors
    neighborhood = ( (-1,-1), (-1, 0), (-1, 1),
                     ( 0,-1),          ( 0, 1),
                     ( 1,-1), ( 1, 0), ( 1, 1) )
    
    def __init__(self, *args, **kwargs):
        'Create the automaton with the specified dimensions.'
        self.dimensions = kwargs.get('dimensions', (50, 50))
        initialdensity = kwargs.get('initialdensity', 0.5)
        # Create a linear grid
        self.cells = [int(random_trial(initialdensity)) for i in range(self.totalcells)]
    # ---
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        alive_neighbors = sum(neighbors)
        # Game of life rules
        if cell:
            return 1 if (1 < alive_neighbors < 4) else 0
        else:
            return 1 if (alive_neighbors == 3) else 0
    # ---
    
    def __str__(self):
        string = ''
        for row in self.state:
            string += ''.join( '#' if cell else '.' for cell in row )
            string += '\n'
        return string
    
    # ---
    
    @property
    def state(self):
        rows, cols = self.dimensions
        state = [ [self.at((i,j)) for j in range(cols) ] for i in range(rows) ]
        return state
    # ---
    
    ### The following is NOT likely to be overriden by subclasses
    
    def neighborsof(self, position):
        'Get the coordinates of the neighboring cells.'
        try:
            # Check if the positions have been already calculated
            return self.neighbors[position]
        
        except AttributeError:
            # Generate neighbors
            self.neighbors = {pos:tuple(self.generate_neighborsof(pos)) 
                               for pos in self.positions}
            return self.neighbors[position]
    # ---
    
    def generate_neighborsof(self, position, log=True):
        'Generate the coordinates of the neighboring cells.'
        # Calculate the neighbors
        for neighbor_pos in self.neighborhood:
            yield tuple((position[i] + neighbor_coord)
                            for i,neighbor_coord in enumerate(neighbor_pos))
    # ---
            
    @property
    def totalcells(self):
        'From the dimensions, calculate the total number of cells.'
        return reduce((lambda x,y: x*y), self.dimensions)
    # ---
    
    def next_position(self, current_position):
        'Fetch the next position from the current one.'
        # It is easier to work with the reverse
        reversed_next = [x for x in reversed(current_position)]
        # Try to increment the current position
        # Starting from the last coordinate
        for i, dim_size in enumerate(reversed(self.dimensions)):
            if reversed_next[i] >= dim_size-1:
                # Cannot increment, reset position
                reversed_next[i] = 0
            else:
                reversed_next[i] += 1
                break
        else:
            # If no position was incremented, 
            return None

        # Flip again to recover the actual next
        return tuple(reversed(reversed_next))
    # ---
    
    @property
    def positions(self):
        try:
            # check if positions have already been gnerated
            return self._positions
        except AttributeError:
            # Generate positions
            self._positions = tuple(self.enumpositions())
            return self.positions

    def enumpositions(self):
        'Enumerate all the positions of the automata.'
        # Initialize...
        current = tuple(0 for dim in self.dimensions)
        while True:
            # Check for a valid position
            if current:
                yield current
            else: 
                break  # No more items
            # Fetch next position
            current = self.next_position(current)
    # ---
    
    def linearize(self, coordinates):
        '''Linearize the given coordinates.
        
        The cells of the automata are stored in a 1D list,
        so, this method returns the linear index for the 
        multidimensional coordinate.
        
        For 2D (rows, cols), the i,j position is:
            j + i*rows.
        
        For 3D (rows, cols, depth) the i,j,k position is:
            k + j*depth + i*cols*depth.
        
        For N dimensions, (D0, D1, D2, ... D[N-1]) the (x0, x1, ... x[N-1]) position is:
            x[N-1] 
            + x[N-2] * D[N-1] 
            + x[N-3] * D[N-2]*D[N-1] 
            + ... 
            + x0 * (D1*D2...*D[N-1])
        '''
        dims = len(self.dimensions)
        # Need to get an item from a 1D array 
        # given it's coordinates as an n dimensional tuple
        multiplier = 1
        linear_position = 0
        for i, dim_size in reversed(list(enumerate(self.dimensions))):
            # Add the weighted coordinate
            linear_position += multiplier*coordinates[i]
            # Calculate the new multiplier
            multiplier *= dim_size
            
        return linear_position
        
    
    def at(self, coordinates):
        '''Get the cell at the given coordinates.
        
        For 2D (rows, cols), the i,j position is:
            j + i*rows.
        
        For 3D (rows, cols, depth) the i,j,k position is:
            k + j*depth + i*cols*depth.
        
        For N dimensions, (D0, D1, D2, ... D[N-1]) the (x0, x1, ... x[N-1]) position is:
            x[N-1] 
            + x[N-2] * D[N-1] 
            + x[N-3] * D[N-2]*D[N-1] 
            + ... 
            + x0 * (D1*D2...*D[N-1])
        '''
        # Auto wrapping of the coordinates
        coordinates = tuple((c+self.dimensions[i]) % self.dimensions[i] 
                                for i,c in enumerate(coordinates))
        
        try:
            return self.cells[self.tolinear[coordinates]]
        except AttributeError:
            # Linear coordinates cache has not been calculated
            self.tolinear = {coords:self.linearize(coords) for coords in self.positions}
            return self.at(coordinates)
        except KeyError as e:
            raise IndexError(f'Coordinates {coordinates} out of range.') from e
    # ---
    
    def step(self):
        'Move one time step forward.'
        self.cells = self.applyrule(self.cells)
    # ---
    
    def applyrule(self, grid):
        'Apply automaton rule to the given grid of cells.'
        # Create a new grid
        newgrid = [cell for cell in self.cells]
        # Apply the rule
        for i,coords in enumerate(self.positions):
            cell_state = self.at(coords)
            cell_neighbors = (self.at(n_coords) 
                                  for n_coords in self.neighborsof(coords))
            # Calculate the new state
            newgrid[i] = self.rule(coords, cell_state, cell_neighbors)
            
        # Return an immutable new grid
        return tuple(newgrid)
    # ---
# --- CellularAutomaton

In [11]:
life = CellularAutomaton()
print(life)
for _ in range(100):
    life.step()
print(life)

##..#..#.###...##.#.####..#.###.......#.##..#.##..
..##..#..#....#.#####.##.##...#...#..#.##.##......
#.#.##.#.....#.#..#.##..######.####..##..##.##.#..
.#.##.##..#......##.###.#.#.#..#...###..#..##..##.
###.##.##.#....#...#.#.#...#.########..#.....#.#..
...#..#.#.#...#############.######.#.###.#.#..###.
.###.###.##....##.#.##...##..#.##.####.###.#...##.
.########..##..#......##...##...###...#.......#.#.
..##.....#....##..#.#.#..#..#..####.###.#....##.#.
..#..#.###..#.#.#..##.###..###..#..##..######...##
..##..#######..##.##.###..##..#...##.....##..#....
.##....###.#....#..#.##..#.#.#.#.###..#.#...#.#.#.
..##...##.#.#...##..#######.#.#.#...#..##.#.......
#.#.#..###.####.#.....###....#...###..#...###.....
#.#.#.#.#....##.###..#.##...#########.###...#..#.#
##..####.##.#.##.#...#...##.######.##....##..##.#.
.##..###..#.#.#.####.####.##.#.##...#.#.######..#.
...#..###.#####.#..##.....#.#..##.###.####.####.##
####.#.#.######..######...###..#.....#.#.....##...
##.#..#.#####.....##...##..#..#

In [12]:
class WolframCA(CellularAutomaton):
    'Instantiation of the CellularAutomata for a 1D automton'
    
    neighborhood = ( (-1,), (0,), (1,))
    
    def __init__(self, length=50, rule=(0, 1, 0, 1, 1, 0, 1, 0)):
        'Create the automaton with the specified dimensions.'
        # Initialize grid
        self.dimensions=(length,)
        self.cells = [0 for _ in range(length)]
        self.cells[length//2] = 1
        # Parse other arguments
        self.ruledef=rule
    # ---
    
    def process_neighborhood(self, neighbors):
        'Convert neighborhood to integer to apply rule.'
        left, current, right = neighbors
        return left + current*2 + right*4
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        i = cell_coords[0]
        grid_dim = self.dimensions[0]
        return self.ruledef[self.process_neighborhood(neighbors)]
    # ---
    
    def __str__(self):
        return ''.join( '#' if cell else '.' for cell in self.state )
    # ---
    
    @property
    def state(self):
        cells = self.dimensions[0]
        state = [ self.at((i,)) for i in range(cells) ]
        return state
    # ---       
    

In [13]:
w = WolframCA(length=100, rule=(0, 1, 1, 0, 1, 0, 0, 0))
for _ in range(50):
    print(w)
    w.step()

..................................................#.................................................
.................................................###................................................
................................................#...#...............................................
...............................................###.###..............................................
..............................................#.......#.............................................
.............................................###.....###............................................
............................................#...#...#...#...........................................
...........................................###.###.###.###..........................................
..........................................#...............#.........................................
.........................................###.............###...............................

In [14]:
class DiffusionCA(CellularAutomaton):
    'Instantiation of the CellularAutomata with a diffusion rule'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.diff_constant = kwargs.get('diff_constant', 0.0001)
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        laplacian = sum(neighbors) - 4*cell
        return cell + self.diff_constant * laplacian
    # ---
    
    def tochar(self, value):
        values = ' .,:;x#X&@EEE '
        return values[int(value / (1/10))-1]
    
    def __str__(self):
        string = ''
        for row in self.state:
            string += ''.join( self.tochar(cell) for cell in row )
            string += '\n'
        return string
    # ---       
    
    

In [15]:
d = DiffusionCA(dimensions=(10, 10), 
                initialdensity=0.4, 
                diff_constant=0.000001)
print(d)

@     @   
  @     @@
@@ @ @    
 @  @@@  @
@ @@@@ @@ 
 @  @     
      @  @
@   @@ @  
 @ @@    @
@     @@@ 



In [16]:
for _ in range(50000): 
    d.step()
print(d)
print(d.state)

&     &. .
  &     X&
@& & &    
.@..E@@  &
& &@@@ && 
 &  &     
      &  X
&   && X  
 & &&    &
&     &&& 

[[0.9339600358052933, 0.14770471200308305, 0.06309293820670048, 0.05678935418098183, 0.01882610519466107, 0.1001617049874685, 0.9252661738581461, 0.22690339424355274, 0.19317764149463, 0.23237601264505023], [0.1963843580166423, 0.1911596607163056, 0.9244974172041268, 0.10143131770270092, 0.10134341942293117, 0.10346316344088514, 0.10653184027394727, 0.10777157543200583, 0.8918155252051294, 0.9731338729979556], [1.0173613559030774, 0.9783742984215208, 0.19542042081579827, 0.9351224792602031, 0.19530474457420918, 0.9739615160511913, 0.15001435035840374, 0.10690582904838945, 0.14800437790023288, 0.1943335205360899], [0.23758221624271716, 1.0185660187889225, 0.23896999838204605, 0.2430928661411247, 1.1029110955431967, 1.062664407714641, 1.0141937539266737, 0.1499695680127101, 0.14893959422487418, 0.9729908252822428], [0.9729880516256795, 0.19639299582780878, 0.9774810361193979, 1.

In [24]:
d = DiffusionCA(dimensions=(100, 100), 
                initialdensity=0.4, 
                diff_constant=0.000001)
#% load_ext line_profiler
%lprun -f d.applyrule for _ in range(10): d.step()

Exploring NumPy
=========

The above automatons showed the structure accomplished the goal of being flexible and to allow not-so-painful implementation of several kinds of CA. The problem is that the performance is too low. 1000 steps on a 10x10 grid of the diffusion automaton took ~10s, and only 10 steps on a 100x100 grid took ~16s. That is a lot!

NumPy offers efficient numeric computations and vectorization, so it looks promising.

In the following we investigate how can we implement some automatons in NumPy.

First, we get into aquaintance of how to do handle Numpy objects

In [23]:
import numpy as np

In [25]:
my_list = [i for i in range(10)]
my_array = np.array(my_list)
my_array

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [27]:
print(my_array[0])
print(my_array[3:5])
print(my_array[-2])
print(my_array[-2:])


0
[3 4]
8
[8 9]


In [28]:
print(my_list + my_list)
print(my_list*2)
print(my_array + my_array)
print(my_array*2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[ 0  2  4  6  8 10 12 14 16 18]
[ 0  2  4  6  8 10 12 14 16 18]


In [30]:
print(np.arange(10))
print(np.arange(1,10))
print(np.arange(1,10,0.5))

[0 1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
[ 1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5  7.   7.5  8.
  8.5  9.   9.5]


In [37]:
print(np.zeros((2,4)))
print(np.ones((10,)))
print(np.ones((3, 3, 3)))

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[[[ 1.  1.  1.]
  [ 1.  1.  1.]
  [ 1.  1.  1.]]

 [[ 1.  1.  1.]
  [ 1.  1.  1.]
  [ 1.  1.  1.]]

 [[ 1.  1.  1.]
  [ 1.  1.  1.]
  [ 1.  1.  1.]]]


In [40]:
help(np.eye)

Help on function eye in module numpy.lib.twodim_base:

eye(N, M=None, k=0, dtype=<class 'float'>)
    Return a 2-D array with ones on the diagonal and zeros elsewhere.
    
    Parameters
    ----------
    N : int
      Number of rows in the output.
    M : int, optional
      Number of columns in the output. If None, defaults to `N`.
    k : int, optional
      Index of the diagonal: 0 (the default) refers to the main diagonal,
      a positive value refers to an upper diagonal, and a negative value
      to a lower diagonal.
    dtype : data-type, optional
      Data-type of the returned array.
    
    Returns
    -------
    I : ndarray of shape (N,M)
      An array where all elements are equal to zero, except for the `k`-th
      diagonal, whose values are equal to one.
    
    See Also
    --------
    identity : (almost) equivalent function
    diag : diagonal 2-D array from a 1-D array specified by the user.
    
    Examples
    --------
    >>> np.eye(2, dtype=int)
    arra

In [42]:
fidentity = np.eye(5, dtype='float16')
bidentity = np.eye(5, dtype='bool')
sidentity = np.eye(5, dtype='unicode_')

print(fidentity, "\n\n")
print(bidentity, "\n\n")
print(sidentity)

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]] 


[[ True False False False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]] 


[['1' '' '' '' '']
 ['' '1' '' '' '']
 ['' '' '1' '' '']
 ['' '' '' '1' '']
 ['' '' '' '' '1']]


In [47]:
a = np.arange(3.)
print(a[0])
print(a[:1])
print(a[::-1])

0.0
[ 0.]
[ 2.  1.  0.]


In [48]:
a = np.eye(3)
print(a[0])
print(a[1][1])
print(a[1,1])

[ 1.  0.  0.]
1.0
1.0


In [52]:
large = np.ones((1000, 1000))
% timeit large[50][100]

374 ns ± 26.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [53]:
% timeit large[50, 100]

216 ns ± 34 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [57]:
arr = np.arange(1, 11, 1)
print(arr*2)
print(arr+6)
print(arr*arr)
print(np.log(arr))

[ 2  4  6  8 10 12 14 16 18 20]
[ 7  8  9 10 11 12 13 14 15 16]
[  1   4   9  16  25  36  49  64  81 100]
[ 0.          0.69314718  1.09861229  1.38629436  1.60943791  1.79175947
  1.94591015  2.07944154  2.19722458  2.30258509]


In [64]:
def generate_array(dimensions=(2,3), value=1):
    'Return an array initialized to the value'
    return np.ones(dimensions)*value

print(generate_array())
print(generate_array((1,2,3), 123456))

[[ 1.  1.  1.]
 [ 1.  1.  1.]]
[[[ 123456.  123456.  123456.]
  [ 123456.  123456.  123456.]]]


In [66]:
def checkerboard(size=10):
    'An nxn matrix with a checkerboard pattern of 1 and 0.'
    m = np.zeros((size,size))
    m[0::2, 0::2] = 1
    m[1::2, 1::2] = 1
    return m

print(checkerboard(5))

[[ 1.  0.  1.  0.  1.]
 [ 0.  1.  0.  1.  0.]
 [ 1.  0.  1.  0.  1.]
 [ 0.  1.  0.  1.  0.]
 [ 1.  0.  1.  0.  1.]]
